In [1]:
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from imutils import build_montages
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [2]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images("dataset"))
data = []
labels = []
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (300,300))
    data.append(image)
    labels.append(label)

[INFO] loading images...


In [3]:
print(set(labels))

{'Loose Silky-bent', 'Cleavers', 'Charlock', 'Common wheat', 'Fat Hen', 'Shepherds Purse', 'Maize', 'Scentless Mayweed', 'Sugar beet', 'Black-grass', 'Common Chickweed', 'Small-flowered Cranesbill'}


In [4]:
data = np.array(data)#, dtype="float") / 255.0
data = data.reshape((data.shape[0], data.shape[1], data.shape[2], 3))
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 12)

In [5]:
print(data.shape, labels.shape)

(4750, 300, 300, 3) (4750, 12)


In [6]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.40, stratify=labels, random_state=42)

In [7]:
opt = Adam(lr=1e-4, decay=1e-4 / 5)

C:\Users\prave\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [8]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [9]:
from tensorflow.keras.applications import EfficientNetB3

model = EfficientNetB3(weights = 'imagenet', include_top=True, input_shape=(300, 300, 3))
output = Dense(12, activation='softmax')(model.layers[-2].output)
model = Model(model.input, output)
model.summary()

50102272/50095040 [==============================] - 10s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 300, 300, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 300, 300, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 301, 301, 3)  0           normalization[0][0]              
____________________________

In [10]:
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [11]:
from tensorflow.keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='./logs',write_graph=True, write_images=False)

In [ ]:
H = model.fit(x=trainX, y=trainY, validation_data=(testX, testY),batch_size=8,  epochs=15, verbose=1) #callbacks=[tensorboard],

Epoch 1/15
357/357 [==============================] - 109s 300ms/step - loss: 1.0750 - accuracy: 0.6702 - val_loss: 0.2377 - val_accuracy: 0.9158
Epoch 2/15
357/357 [==============================] - 104s 292ms/step - loss: 0.2926 - accuracy: 0.9095 - val_loss: 0.1578 - val_accuracy: 0.9453
Epoch 3/15
357/357 [==============================] - 105s 293ms/step - loss: 0.1778 - accuracy: 0.9467 - val_loss: 0.1500 - val_accuracy: 0.9521
Epoch 4/15
357/357 [==============================] - 104s 293ms/step - loss: 0.1275 - accuracy: 0.9618 - val_loss: 0.1376 - val_accuracy: 0.9611
Epoch 5/15
357/357 [==============================] - 105s 294ms/step - loss: 0.0797 - accuracy: 0.9779 - val_loss: 0.1312 - val_accuracy: 0.9616
Epoch 6/15
357/357 [==============================] - 105s 294ms/step - loss: 0.0549 - accuracy: 0.9842 - val_loss: 0.1329 - val_accuracy: 0.9611
Epoch 7/15
357/357 [==============================] - 105s 293ms/step - loss: 0.0555 - accuracy: 0.9856 - val_loss: 0.1529 -

In [ ]:
model.save("plantclass.h5")

In [ ]:
y_pred = model.predict(testX)

In [ ]:
def plot_confusion_matrix(test_y, predict_y):
    C = confusion_matrix(test_y, predict_y)   
    A =(((C.T)/(C.sum(axis=1))).T)    
    B =(C/C.sum(axis=0))
    plt.figure(figsize=(20,4))    
    labels = [0,1]
    cmap=sns.light_palette("blue")    
    plt.subplot(1, 3, 2)
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Precision matrix")    
    plt.subplot(1, 3, 3)
    # representing B in heatmap format
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Recall matrix")    
    plt.show()

In [ ]:
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(testY, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

plot_confusion_matrix(y_test, y_pred)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model

model = load_model("plantclass.h5")
#model.summary()
IMAGE_PATH = 'test/6edc76e7c.png'
LAYER_NAME = 'top_conv'

img =cv2.imread(IMAGE_PATH)
img1 = cv2.resize(img, (300,300))
img2 = np.expand_dims(img1,axis=0)

i = np.argmax(model.predict(img2)[0])
grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(LAYER_NAME).output, model.output])

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([img1]))
    loss = predictions[:,0]

output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]  * 1e+08
gate_f = tf.cast(output > 0, 'float32')
gate_r = tf.cast(grads > 0, 'float32')
guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

weights = tf.reduce_mean(guided_grads, axis=(0, 1))

cam = np.ones(output.shape[0: 2], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, :, i]

cam = cv2.resize(cam.numpy(), (img.shape[1], img.shape[0]))
denom = (cam.max() - cam.min()) + 1e-16
cam = np.maximum(cam, 0)
heatmap = (cam - cam.min()) / denom
cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 1, 0)
plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))